Part e

In [3]:
"This notebook consist of the own Feed Fordward Neural Network. This code dervies from the weekly exercise from week 42, and lecture notes from week 38, 39, 40 and 41."

# Importing necesscary libaries
import autograd.numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
import matplotlib.pyplot as plt
from autograd import grad

np.random.seed(20)

In [4]:
"This section loads the data, splits it into training and test-dara, and scales it using Scikit-learn StandardScalar"
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# This adds a cloum for the bias
X_train_scaled = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
X_test_scaled = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]

In [5]:
#Defines the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Defines the binary cross-entropy cost
def binary_cross_entropy(theta, X, y, lambda_reg=0.01):
    n = len(y)
    predictions = sigmoid(X @ theta)
    cost = -(1/n) * np.sum(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    l2_penalty = lambda_reg * np.sum(theta[1:]**2)  # to avoid that the bias term gets regulated
    return cost + l2_penalty

In [ ]:
# Using Autograd to calculate the gradients
gradient_func = grad(binary_cross_entropy)

# Defines the function for the logistic regression with SGD
def logistic_regression_sgd(X, y, n_epochs=500, batch_size=16, lambda_reg=0.01, momentum=0.9, delta=1e-6):
    m, n= X.shape
    theta = np.random.randn(n)  # Initialize weights randomly
    t0, t1 = 5, 50  # Parameters for learning schedule

    change = np.zeros_like(theta)
    Giter = np.zeros_like(theta)
    def learning_schedule(t):
        return t0 / (t + t1)

    # Run mini-batch SGD, from weekly exercise week 41
    for epoch in range(n_epochs):
        indices = np.random.permutation(m)
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        for i in range(0, m, batch_size):
            xi = X_shuffled[i:i + batch_size]
            yi = y_shuffled[i:i + batch_size]
            gradients = gradient_func(theta, xi, yi, lambda_reg)

            Giter += gradients*gradients
            adjusted_gradients = gradients/(delta + np.sqrt(Giter))
            eta = learning_schedule(epoch *(m//batch_size)+i//batch_size)

            new_change = momentum*change - eta*adjusted_gradients
            theta += new_change
            change = new_change

    return theta

# Using the sigmoid function to return the probability
def predict_logistic_regression(X, theta):
    y_pred_proba = 1/ (1+np.exp(-X @ theta))
    return y_pred_proba

In [ ]:
# Calculate teta, probability score, accuracy ans loss. 
theta_sgd = logistic_regression_sgd(X_train_scaled, y_train, n_epochs=500, batch_size=16, lambda_reg=0.01)
y_test_prob = predict_logistic_regression(X_test_scaled, theta_sgd)
pred_sgd =(y_test_prob >=0.5).astype(int)

accuracy_sgd = accuracy_score(y_test, pred_sgd)
print(f"Accuracy {accuracy_sgd}")
test_loss_sgd = log_loss(y_test,y_test_prob)
print(f"Test loss {test_loss_sgd}")

Accuracy 0.9736842105263158
Test loss 0.1072396307747428


Using Scikit-learn

In [ ]:
"The main part of this code dervie from lecture notes week 38."

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

log_reg = LogisticRegression(penalty="l2",random_state=1, max_iter=500)  

log_reg.fit(X_train_scaled, y_train)

proba_sklearn = log_reg.predict_proba(X_test_scaled)[:,1]
pred_sklearn = (proba_sklearn >=0.5).astype(int)

accuracy_sklearn = accuracy_score(y_test, pred_sklearn)
test_loss_sklearn = log_loss(y_test, proba_sklearn)

print(f"Scikit-Learn Logistic Regression Accuracy: {accuracy_sklearn:.4f}")
print(f"Scikit-Learn Logistic test loss: {test_loss_sklearn:.4f}")

Scikit-Learn Logistic Regression Accuracy: 0.9737
Scikit-Learn Logistic test loss: 0.0718
